---
title: "Turn-Based vs Time-Based: Synchronization Strategies"
description: "Exploring different approaches to coordinating multiple agents in a spatial environment"
author: "Eric Zou"
date: "12/12/2025"
categories:
  - Agents
  - Simulation
  - Infrastructure
---


# The Synchronization Problem

When you have multiple agents (and eventually humans) acting in the same 2D space, you need to decide: **how do they act relative to each other?**

In previous posts, we've been using a simple sequential approach—agents take turns one after another. But as we scale up and prepare for human-AI interaction experiments, we need to think more carefully about synchronization strategies.

This post explores two fundamental approaches:
1. **Turn-Based**: Agents act sequentially in a fixed order
2. **Time-Based**: Agents act concurrently within discrete time steps

Each has different implications for behavior, fairness, and realism.


## Turn-Based Approach

In a turn-based system, agents act one at a time in a fixed sequence. This is what we've been doing so far.

### Characteristics:
- **Deterministic**: Same order every time
- **Sequential**: Each agent sees the world state after the previous agent's action
- **Simple**: Easy to reason about and debug
- **Fair**: Every agent gets exactly one action per "round"

### Implementation


In [45]:
import os
import re
import random
import time
from dataclasses import dataclass, field
from typing import List, Optional
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv("../../.env")
client = OpenAI()

GRID_SIZE = 20

@dataclass
class Agent:
    name: str
    x: int
    y: int
    color: str
    history: list = field(default_factory=list)
    
    def move(self, dx, dy):
        self.x = max(0, min(GRID_SIZE-1, self.x + dx))
        self.y = max(0, min(GRID_SIZE-1, self.y + dy))

def run_turn_based_round(agents, shared_transcript):
    """
    Turn-based: Each agent acts sequentially in a fixed order.
    Each agent sees the world state AFTER all previous agents have acted.
    """
    for agent in agents:
        # Agent sees current world state (which includes previous agents' actions)
        others = [a for a in agents if a != agent]
        others_loc = "\n".join([f"- {a.name}: ({a.x}, {a.y})" for a in others])
        
        system_prompt = f"""
You have just joined an online multiplayer chatroom as an avatar in a 2D grid. Discuss any topic, including those beyond the grid.

You are {agent.name}, positioned at ({agent.x}, {agent.y}) in a 20x20 grid.

Other avatars currently visible:
{others_loc}

Recent chat messages:
{chr(10).join(shared_transcript[-3:]) if shared_transcript else "No messages yet."}

You can do BOTH:
1. Move your avatar using [MOVE: DIRECTION] (UP, DOWN, LEFT, RIGHT)
2. Chat about anything - the grid, your position, or any topic you want

You can move and speak in the same turn. Format: [MOVE: DIRECTION] followed by your message, or just speak without moving.

Keep your response short (1-2 sentences).
"""
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": system_prompt}]
        )
        content = response.choices[0].message.content.strip()
        
        # Parse and execute actions - can do both move and speak
        match = re.search(r'\[MOVE:\s*(\w+)\]', content)
        if match:
            direction = match.group(1).upper()
            if direction == "UP": agent.move(0, 1)
            elif direction == "DOWN": agent.move(0, -1)
            elif direction == "LEFT": agent.move(-1, 0)
            elif direction == "RIGHT": agent.move(1, 0)
        
        # Extract message (remove MOVE command if present)
        message = re.sub(r'\[MOVE:\s*\w+\]', '', content).strip()
        if message:
            shared_transcript.append(f"{agent.name}: {message}")
        
        # World state has now changed for the next agent
        print(f"{agent.name} acted. World state updated.")


In [ ]:
# Example: Run turn-based rounds
agents = [
    Agent("Alice", 5, 5, "red"),
    Agent("Bob", 15, 15, "blue")
]

transcript = []

print("=== Turn-Based Rounds ===")
print(f"Initial positions: Alice ({agents[0].x}, {agents[0].y}), Bob ({agents[1].x}, {agents[1].y})")
print()

num_rounds = 10
for i in range(num_rounds):
    print(f"Round {i+1}:")
    run_turn_based_round(agents, transcript)
    print(f"  Positions: Alice ({agents[0].x}, {agents[0].y}), Bob ({agents[1].x}, {agents[1].y})")
    print()

print(f"Final positions: Alice ({agents[0].x}, {agents[0].y}), Bob ({agents[1].x}, {agents[1].y})")
print(f"\nTotal messages: {len(transcript)}")
print("\nTranscript:")
for msg in transcript:
    print(f"  {msg}")


=== Turn-Based Rounds ===
Initial positions: Alice (5, 5), Bob (15, 15)

Round 1:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (5, 5), Bob (15, 15)

Round 2:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (6, 5), Bob (14, 15)

Round 3:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (7, 5), Bob (13, 15)

Round 4:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (8, 5), Bob (12, 15)

Round 5:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (8, 6), Bob (11, 15)

Round 6:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (8, 7), Bob (10, 15)

Round 7:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (8, 8), Bob (9, 15)

Round 8:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (9, 8), Bob (8, 15)

R

### Pros and Cons

**Advantages:**
- Predictable and deterministic
- Easy to implement and debug
- No race conditions
- Fair (everyone gets equal turns)
- Good for turn-based games or strategic interactions

**Disadvantages:**
- Order matters: first agent has advantage
- Not realistic for real-time scenarios
- Can feel artificial to humans
- Sequential processing is slow for many agents
- Agents later in the order see a different world state


## Time-Based Approach

In a time-based system, all agents act "simultaneously" within discrete time steps. Actions are collected first, then applied all at once.

### Characteristics:
- **Concurrent**: All agents decide actions based on the same world state
- **Batch Processing**: Actions collected, then applied together
- **More Realistic**: Mimics real-time behavior
- **Parallelizable**: Can generate actions in parallel

### Implementation


In [47]:
@dataclass
class Action:
    agent: Agent
    direction: Optional[str] = None  # Move direction if moving
    message: Optional[str] = None  # Message if speaking

def get_agent_action(agent: Agent, all_agents: List[Agent], shared_transcript: List[str]) -> Action:
    """
    Generate an action for a single agent based on CURRENT world state.
    This can be called in parallel for all agents.
    """
    others = [a for a in all_agents if a != agent]
    others_loc = "\n".join([f"- {a.name}: ({a.x}, {a.y})" for a in others])
    
    system_prompt = f"""
You have just joined an online multiplayer chatroom as an avatar in a 2D grid. Discuss any topic, including those beyond the grid.

You are {agent.name}, positioned at ({agent.x}, {agent.y}) in a 20x20 grid.

Other avatars currently visible:
{others_loc}

Recent chat messages:
{chr(10).join(shared_transcript[-3:]) if shared_transcript else "No messages yet."}

IMPORTANT: All avatars are deciding their actions simultaneously. You don't know what others will do this turn.

You can do BOTH:
1. Move your avatar using [MOVE: DIRECTION] (UP, DOWN, LEFT, RIGHT)
2. Chat about anything - the grid, your position, or any topic you want

You can move and speak in the same turn. Format: [MOVE: DIRECTION] followed by your message, or just speak without moving.

Keep your response short (1-2 sentences).
"""
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": system_prompt}]
    )
    content = response.choices[0].message.content.strip()
    
    # Parse actions - can do both move and speak
    direction = None
    match = re.search(r'\[MOVE:\s*(\w+)\]', content)
    if match:
        direction = match.group(1).upper()
    
    # Extract message (remove MOVE command if present)
    message = re.sub(r'\[MOVE:\s*\w+\]', '', content).strip()
    if not message:
        message = None
    
    return Action(agent, direction=direction, message=message)

def run_time_based_round(agents: List[Agent], shared_transcript: List[str]):
    """
    Time-based: All agents decide actions simultaneously based on the same world state.
    Actions are then applied all at once.
    """
    # Phase 1: Collect actions (can be parallelized)
    actions = []
    
    # Sequential version (for now)
    for agent in agents:
        action = get_agent_action(agent, agents, shared_transcript)
        actions.append(action)
    
    # Phase 2: Apply all actions
    for action in actions:
        # Apply move if present
        if action.direction:
            direction = action.direction
            if direction == "UP": action.agent.move(0, 1)
            elif direction == "DOWN": action.agent.move(0, -1)
            elif direction == "LEFT": action.agent.move(-1, 0)
            elif direction == "RIGHT": action.agent.move(1, 0)
        
        # Apply message if present
        if action.message:
            shared_transcript.append(f"{action.agent.name}: {action.message}")
    
    print(f"Applied {len(actions)} actions simultaneously.")


### Parallel Version

Since agents decide independently, we can parallelize action generation:


In [ ]:
# Example: Run time-based rounds
agents_time = [
    Agent("Alice", 5, 5, "red"),
    Agent("Bob", 15, 15, "blue")
]

transcript_time = []

print("=== Time-Based Rounds ===")
print(f"Initial positions: Alice ({agents_time[0].x}, {agents_time[0].y}), Bob ({agents_time[1].x}, {agents_time[1].y})")
print()

num_rounds = 10
for i in range(num_rounds):
    print(f"Round {i+1}:")
    run_time_based_round(agents_time, transcript_time)
    print(f"  Positions: Alice ({agents_time[0].x}, {agents_time[0].y}), Bob ({agents_time[1].x}, {agents_time[1].y})")
    print()

print(f"Final positions: Alice ({agents_time[0].x}, {agents_time[0].y}), Bob ({agents_time[1].x}, {agents_time[1].y})")
print(f"\nTotal messages: {len(transcript_time)}")
print("\nTranscript:")
for msg in transcript_time:
    print(f"  {msg}")


=== Time-Based Rounds ===
Initial positions: Alice (5, 5), Bob (15, 15)

Round 1:
Applied 2 actions simultaneously.
  Positions: Alice (5, 6), Bob (15, 16)

Round 2:
Applied 2 actions simultaneously.
  Positions: Alice (5, 7), Bob (15, 15)

Round 3:
Applied 2 actions simultaneously.
  Positions: Alice (5, 8), Bob (16, 15)

Round 4:
Applied 2 actions simultaneously.
  Positions: Alice (5, 9), Bob (17, 15)

Round 5:
Applied 2 actions simultaneously.
  Positions: Alice (5, 10), Bob (16, 15)

Round 6:
Applied 2 actions simultaneously.
  Positions: Alice (5, 11), Bob (17, 15)

Round 7:
Applied 2 actions simultaneously.
  Positions: Alice (4, 11), Bob (16, 15)

Round 8:
Applied 2 actions simultaneously.
  Positions: Alice (4, 12), Bob (17, 15)

Round 9:
Applied 2 actions simultaneously.
  Positions: Alice (4, 11), Bob (17, 16)

Round 10:
Applied 2 actions simultaneously.
  Positions: Alice (4, 12), Bob (17, 17)

Final positions: Alice (4, 12), Bob (17, 17)
Total messages: 20


In [49]:
from concurrent.futures import ThreadPoolExecutor

def run_time_based_parallel(agents: List[Agent], shared_transcript: List[str], max_workers: int = 4):
    """
    Time-based with parallel action generation.
    """
    # Phase 1: Collect actions in parallel
    actions = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(get_agent_action, agent, agents, shared_transcript) 
                   for agent in agents]
        actions = [future.result() for future in futures]
    
    # Phase 2: Apply all actions (must be sequential to maintain consistency)
    for action in actions:
        # Apply move if present
        if action.direction:
            direction = action.direction
            if direction == "UP": action.agent.move(0, 1)
            elif direction == "DOWN": action.agent.move(0, -1)
            elif direction == "LEFT": action.agent.move(-1, 0)
            elif direction == "RIGHT": action.agent.move(1, 0)
        
        # Apply message if present
        if action.message:
            shared_transcript.append(f"{action.agent.name}: {action.message}")
    
    print(f"Generated {len(actions)} actions in parallel, then applied sequentially.")


In [ ]:
# Example: Compare turn-based vs time-based over multiple rounds
import time as time_module

print("=== COMPARISON: Turn-Based vs Time-Based ===\n")

# Reset agents
agents_tb = [Agent("Alice", 5, 5, "red"), Agent("Bob", 15, 15, "blue")]
agents_tm = [Agent("Alice", 5, 5, "red"), Agent("Bob", 15, 15, "blue")]
transcript_tb = []
transcript_tm = []

num_rounds = 10

print("TURN-BASED:")
for i in range(num_rounds):
    print(f"\nRound {i+1}:")
    run_turn_based_round(agents_tb, transcript_tb)
    print(f"  Positions: Alice ({agents_tb[0].x}, {agents_tb[0].y}), Bob ({agents_tb[1].x}, {agents_tb[1].y})")

print(f"\nFinal: Alice ({agents_tb[0].x}, {agents_tb[0].y}), Bob ({agents_tb[1].x}, {agents_tb[1].y})")
print(f"\nTurn-based transcript ({len(transcript_tb)} messages):")
for msg in transcript_tb:
    print(f"  {msg}")

print("\n" + "="*50)
print("TIME-BASED:")

# Reset for time-based
agents_tm = [Agent("Alice", 5, 5, "red"), Agent("Bob", 15, 15, "blue")]
transcript_tm = []

for i in range(num_rounds):
    print(f"\nRound {i+1}:")
    run_time_based_round(agents_tm, transcript_tm)
    print(f"  Positions: Alice ({agents_tm[0].x}, {agents_tm[0].y}), Bob ({agents_tm[1].x}, {agents_tm[1].y})")

print(f"\nFinal: Alice ({agents_tm[0].x}, {agents_tm[0].y}), Bob ({agents_tm[1].x}, {agents_tm[1].y})")
print(f"\nTime-based transcript ({len(transcript_tm)} messages):")
for msg in transcript_tm:
    print(f"  {msg}")


=== COMPARISON: Turn-Based vs Time-Based ===

TURN-BASED:

Round 1:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (5, 5), Bob (14, 15)

Round 2:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (5, 6), Bob (13, 15)

Round 3:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (6, 6), Bob (12, 15)

Round 4:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (7, 6), Bob (11, 15)

Round 5:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (8, 6), Bob (10, 15)

Round 6:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (9, 6), Bob (10, 16)

Round 7:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (9, 7), Bob (9, 16)

Round 8:
Alice acted. World state updated.
Bob acted. World state updated.
  Positions: Alice (9, 8), Bob (9, 17)

Round 9:
Alice 

### Pros and Cons

**Advantages:**
- More realistic (simulates simultaneous action)
- Fair (all agents see same world state)
- Parallelizable (faster for many agents)
- Better for real-time feel
- No order bias

**Disadvantages:**
- More complex to implement
- Potential for conflicts (multiple agents want same space)
- Less deterministic (though still reproducible if seeded)
- Requires conflict resolution strategies
- Agents can't react to others' actions in the same turn


## Comparison: Behavioral Differences

The choice of synchronization strategy fundamentally changes how agents behave:

### Turn-Based Example
```
Round 1:
  Alice (at 5,5) sees Bob at (10,10) → moves RIGHT
  Bob (at 10,10) sees Alice at (6,5) → moves LEFT (reacts to Alice's move)
  
Result: Agents can react to each other within the same round
```

### Time-Based Example
```
Round 1:
  Alice (at 5,5) sees Bob at (10,10) → decides to move RIGHT
  Bob (at 10,10) sees Alice at (5,5) → decides to move LEFT
  Both actions applied simultaneously
  
Result: Agents act on incomplete information (don't see others' moves)
```

This creates different emergent behaviors:


In [ ]:
# Example: Hybrid approach with random order
agents_hybrid = [
    Agent("Alice", 5, 5, "red"),
    Agent("Bob", 15, 15, "blue"),
    Agent("Charlie", 10, 10, "green")
]

transcript_hybrid = []

print("=== Hybrid: Random Order Each Round ===")
print(f"Initial positions:")
for a in agents_hybrid:
    print(f"  {a.name}: ({a.x}, {a.y})")
print()

num_rounds = 10
for i in range(num_rounds):
    print(f"Round {i+1}:")
    run_hybrid_random_order(agents_hybrid, transcript_hybrid)
    print(f"  Positions: ", end="")
    for a in agents_hybrid:
        print(f"{a.name} ({a.x}, {a.y}) ", end="")
    print()
print()

print(f"Final positions:")
for a in agents_hybrid:
    print(f"  {a.name}: ({a.x}, {a.y})")
print(f"\nTotal messages: {len(transcript_hybrid)}")
print("\nTranscript:")
for msg in transcript_hybrid:
    print(f"  {msg}")


=== Hybrid: Random Order Each Round ===
Initial positions:
  Alice: (5, 5)
  Bob: (15, 15)
  Charlie: (10, 10)

Round 1:
  Positions: Alice (5, 6) Bob (15, 15) Charlie (10, 11) 
Round 2:
  Positions: Alice (5, 7) Bob (14, 15) Charlie (10, 12) 
Round 3:
  Positions: Alice (5, 8) Bob (13, 15) Charlie (10, 13) 
Round 4:
  Positions: Alice (5, 9) Bob (12, 15) Charlie (11, 13) 
Round 5:
  Positions: Alice (6, 9) Bob (11, 15) Charlie (11, 14) 
Round 6:
  Positions: Alice (6, 10) Bob (10, 15) Charlie (11, 15) 
Round 7:
  Positions: Alice (7, 10) Bob (9, 15) Charlie (10, 15) 
Round 8:
  Positions: Alice (7, 11) Bob (9, 16) Charlie (9, 15) 
Round 9:
  Positions: Alice (7, 12) Bob (8, 16) Charlie (9, 16) 
Round 10:
  Positions: Alice (7, 13) Bob (8, 17) Charlie (8, 16) 

Final positions:
  Alice: (7, 13)
  Bob: (8, 17)
  Charlie: (8, 16)
Total messages: 30


## Hybrid Approaches

You can also combine elements of both:

### 1. Turn-Based with Random Order
Shuffle the agent order each round to reduce first-mover advantage.

### 2. Time-Based with Action Phases
Split actions into phases (e.g., "planning" then "execution") where agents can see others' plans before final execution.

### 3. Continuous Time with Tick Rate
Agents can act at different rates (some faster, some slower), creating more naturalistic behavior.

### 4. Event-Driven
Agents act in response to events (messages, proximity changes) rather than fixed time steps.


In [52]:
# Example: Time-based with parallel execution (if you have multiple agents)
# Note: This requires multiple API calls, so it's more expensive but faster

agents_parallel = [
    Agent("Alice", 5, 5, "red"),
    Agent("Bob", 15, 15, "blue"),
    Agent("Charlie", 10, 10, "green"),
    Agent("Dave", 0, 0, "orange")
]

transcript_parallel = []

print("=== Time-Based with Parallel Execution ===")
print(f"Initial positions:")
for a in agents_parallel:
    print(f"  {a.name}: ({a.x}, {a.y})")
print()

import time as time_module
start = time_module.time()
num_rounds = 10
for i in range(num_rounds):
    print(f"Round {i+1}:")
    run_time_based_parallel(agents_parallel, transcript_parallel, max_workers=4)
    print(f"  Positions: ", end="")
    for a in agents_parallel:
        print(f"{a.name} ({a.x}, {a.y}) ", end="")
    print()
end = time_module.time()

print()
print(f"Final positions:")
for a in agents_parallel:
    print(f"  {a.name}: ({a.x}, {a.y})")
print(f"Total time taken: {end - start:.2f} seconds")
print(f"\nTotal messages: {len(transcript_parallel)}")
print("\nTranscript:")
for msg in transcript_parallel:
    print(f"  {msg}")


=== Time-Based with Parallel Execution ===
Initial positions:
  Alice: (5, 5)
  Bob: (15, 15)
  Charlie: (10, 10)
  Dave: (0, 0)

Round 1:


Generated 4 actions in parallel, then applied sequentially.
  Positions: Alice (6, 5) Bob (14, 15) Charlie (10, 11) Dave (0, 1) 
Round 2:
Generated 4 actions in parallel, then applied sequentially.
  Positions: Alice (5, 5) Bob (13, 15) Charlie (10, 12) Dave (1, 1) 
Round 3:
Generated 4 actions in parallel, then applied sequentially.
  Positions: Alice (5, 6) Bob (12, 15) Charlie (10, 13) Dave (2, 1) 
Round 4:
Generated 4 actions in parallel, then applied sequentially.
  Positions: Alice (5, 7) Bob (11, 15) Charlie (10, 14) Dave (3, 1) 
Round 5:
Generated 4 actions in parallel, then applied sequentially.
  Positions: Alice (5, 8) Bob (10, 15) Charlie (9, 14) Dave (4, 1) 
Round 6:
Generated 4 actions in parallel, then applied sequentially.
  Positions: Alice (4, 8) Bob (9, 15) Charlie (8, 14) Dave (5, 1) 
Round 7:
Generated 4 actions in parallel, then applied sequentially.
  Positions: Alice (4, 9) Bob (8, 15) Charlie (7, 14) Dave (6, 1) 
Round 8:
Generated 4 actions in parallel, then a

In [53]:
def run_hybrid_random_order(agents: List[Agent], shared_transcript: List[str]):
    """
    Turn-based but with random order each round.
    """
    shuffled = agents.copy()
    random.shuffle(shuffled)
    
    for agent in shuffled:
        # Same logic as turn-based, but order changes
        others = [a for a in agents if a != agent]
        others_loc = "\n".join([f"- {a.name}: ({a.x}, {a.y})" for a in others])
        
        system_prompt = f"""
You have just joined an online multiplayer chatroom as an avatar in a 2D grid. Discuss any topic, including those beyond the grid.

You are {agent.name}, positioned at ({agent.x}, {agent.y}) in a 20x20 grid.

Other avatars currently visible:
{others_loc}

Recent chat messages:
{chr(10).join(shared_transcript[-3:]) if shared_transcript else "No messages yet."}

You can do BOTH:
1. Move your avatar using [MOVE: DIRECTION] (UP, DOWN, LEFT, RIGHT)
2. Chat about anything - the grid, your position, or any topic you want

You can move and speak in the same turn. Format: [MOVE: DIRECTION] followed by your message, or just speak without moving.

Keep your response short (1-2 sentences).
"""
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": system_prompt}]
        )
        content = response.choices[0].message.content.strip()
        
        # Parse and execute actions - can do both move and speak
        match = re.search(r'\[MOVE:\s*(\w+)\]', content)
        if match:
            direction = match.group(1).upper()
            if direction == "UP": agent.move(0, 1)
            elif direction == "DOWN": agent.move(0, -1)
            elif direction == "LEFT": agent.move(-1, 0)
            elif direction == "RIGHT": agent.move(1, 0)
        
        # Extract message (remove MOVE command if present)
        message = re.sub(r'\[MOVE:\s*\w+\]', '', content).strip()
        if message:
            shared_transcript.append(f"{agent.name}: {message}")


## Implications for Human-AI Experiments

For the experiment proposed in post 016, we need to consider:

### Turn-Based Advantages:
- **Easier for humans**: Familiar from board games
- **Predictable**: Humans can plan ahead
- **Fair**: No advantage based on network latency
- **Debuggable**: Clear sequence of events

### Time-Based Advantages:
- **More natural**: Feels more like real-time interaction
- **Scalable**: Can handle many agents efficiently with parallelization
- **Realistic**: Mimics how people actually interact in spaces
- **No order bias**: All participants see the same state

### Recommendation for Experiment:

For the human-AI interaction study, I'd suggest **time-based with discrete ticks**:

1. **Fixed tick rate** (e.g., 1 action per 5 seconds)
2. **All agents decide simultaneously** based on current state
3. **Actions applied together** at tick boundary
4. **Humans submit actions** before tick deadline
5. **AI agents generate actions** in parallel

This balances:
- Realism (feels real-time)
- Fairness (everyone sees same state)
- Scalability (parallel AI generation)
- Human usability (clear time boundaries)

### Implementation Considerations:

- **Action deadlines**: Humans need time to read and respond
- **Latency handling**: What if human doesn't submit in time?
- **Conflict resolution**: If multiple agents want same space (we'll leave this to agent behavior as discussed)
- **State snapshots**: Need to capture world state at each tick for analysis
- **Replay capability**: Record all actions for later analysis


## Conclusion

The choice between turn-based and time-based synchronization isn't just a technical detail—it fundamentally shapes how agents (and humans) interact in the space.

**Turn-based** is simpler and more strategic, but can feel artificial.

**Time-based** is more realistic and scalable, but requires careful handling of conflicts and state consistency.

For our upcoming human-AI experiments, a **time-based approach with discrete ticks** seems like the best balance of realism, fairness, and practicality.
